In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.count()

7778101

In [4]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [5]:
df_green.registerTempTable('green')

/usr/local/spark/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [6]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2019-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2019-01-01 00:00:00|   3|             90.02|             2|
|2019-01-01 00:00:00|   7| 847.5499999999993|            68|
|2019-01-01 00:00:00|  11|              10.3|             1|
|2019-01-01 00:00:00|  14|              77.4|             3|
|2019-01-01 00:00:00|  16|50.599999999999994|             2|
|2019-01-01 00:00:00|  17|170.85000000000002|            10|
|2019-01-01 00:00:00|  18|             75.71|             3|
|2019-01-01 00:00:00|  20|             47.55|             1|
|2019-01-01 00:00:00|  24|              75.5|             6|
|2019-01-01 00:00:00|  25|1161.3399999999995|            48|
|2019-01-01 00:00:00|  26|              21.3|             1|
|2019-01-01 00:00:00|  28|             21.51|             1|
|2019-01-01 00:00:00|  29|              22.1|             2|
|2019-01-01 00:00:00|  3

In [ ]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [ ]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [ ]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2019-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [ ]:
df_yellow_revenue.show()

In [ ]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [8]:
df_green_years = spark.sql("""
SELECT 
    date_trunc('year', lpep_pickup_datetime) AS year,
    COUNT(*) AS number_records
FROM
    green

GROUP BY
    1
""")

In [9]:
df_green_years.show()

+-------------------+--------------+
|               year|number_records|
+-------------------+--------------+
|2019-01-01 00:00:00|       6043796|
|2009-01-01 00:00:00|           121|
|2018-01-01 00:00:00|            67|
|2008-01-01 00:00:00|            20|
|2010-01-01 00:00:00|            55|
|2062-01-01 00:00:00|             1|
|2020-01-01 00:00:00|       1734038|
|2035-01-01 00:00:00|             1|
|2021-01-01 00:00:00|             1|
|2041-01-01 00:00:00|             1|
+-------------------+--------------+



In [ ]:
df_yellow_years = spark.sql("""
SELECT 
    date_trunc('year', tpep_pickup_datetime) AS year,
    COUNT(*) AS number_records
FROM
    yellow

GROUP BY
    1
""")

In [ ]:
df_yellow_years.show()

In [ ]:
break

In [ ]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [ ]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [ ]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [ ]:
df_join = spark.read.parquet('data/report/revenue/total')

In [ ]:
df_join

In [ ]:
df_zones = spark.read.parquet('zones/')

In [ ]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [ ]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')